In [100]:
# 主要モジュール
import torch
import pandas as pd
# import numpy as np
import matplotlib.pyplot as plt

# 学習用のサンプルデータ
from sklearn import datasets

# データを整理する
from torch.utils.data import DataLoader

# 全結合層と活性化関数
from torch import nn
from torch.nn import functional as F

# 損失関数と最適化関数
from torch import optim
import pandas as pd
import numpy as np
from datetime import datetime, timedelta


In [101]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"run with {device}")

run with cuda


In [102]:
df = pd.read_csv('../database/normaraze_stock_price/dataset_20231026.csv')

In [103]:
df

,name,2023-07-26,2023-07-27,2023-07-28,2023-07-31,2023-08-01,2023-08-02,2023-08-03,2023-08-04,2023-08-07,...,2023-10-12,2023-10-13,2023-10-16,2023-10-17,2023-10-18,2023-10-19,2023-10-20,2023-10-23,2023-10-24,2023-10-25
0,AACI,10.570,10.5700,10.590000,10.570,10.500,10.5600,10.570,10.500,10.5000,...,10.630,10.69,10.720,10.690,10.690,10.690,10.690,10.690,10.690,NaN
1,AADI,5.760,5.3800,5.360000,5.330,5.250,5.2600,5.660,5.730,5.9900,...,4.200,4.23,4.220,4.180,3.990,3.960,4.090,3.940,4.290,4.2200
2,AAL,16.760,16.5700,16.790001,16.750,16.230,15.9500,15.980,15.840,15.9900,...,12.060,11.72,11.940,11.940,11.360,11.450,11.080,11.290,11.210,11.0400
3,AAME,1.940,1.8700,1.830000,1.830,1.770,1.7600,1.810,1.920,1.8600,...,1.860,1.85,1.820,1.730,1.800,1.930,1.810,1.940,1.940,1.8000
4,AAOI,7.950,6.1900,6.710000,6.750,7.030,7.3900,6.590,11.010,12.3800,...,7.950,7.55,7.650,7.590,7.550,7.450,6.910,7.170,7.440,7.1200
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2947,ZVRA,4.710,4.6400,4.830000,4.880,4.900,4.8100,4.840,4.820,4.8600,...,4.560,4.47,4.460,4.530,4.370,4.370,4.210,4.290,4.370,4.3900
2948,ZVSA,0.166,0.1620,0.169000,0.173,0.170,0.1700,0.160,0.170,0.1640,...,0.131,0.13,0.126,0.122,0.128,0.128,0.125,0.118,0.116,0.1145
2949,ZYME,7.510,7.2800,7.270000,7.460,7.450,7.3200,7.320,7.390,7.2600,...,6.420,6.50,7.050,7.250,7.160,7.210,7.190,7.000,7.010,6.9700
2950,ZYNE,0.364,0.3551,0.359800,0.365,0.355,0.3688,0.352,0.353,0.3421,...,1.300,1.30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [104]:
from sklearn.preprocessing import LabelEncoder

# ラベルを数値に変換する
le = LabelEncoder()
df['encode_name'] = le.fit_transform(df['name'])
df.insert(1, 'encode_name', df.pop('encode_name'))

In [105]:
df

,name,encode_name,2023-07-26,2023-07-27,2023-07-28,2023-07-31,2023-08-01,2023-08-02,2023-08-03,2023-08-04,...,2023-10-12,2023-10-13,2023-10-16,2023-10-17,2023-10-18,2023-10-19,2023-10-20,2023-10-23,2023-10-24,2023-10-25
0,AACI,0,10.570,10.5700,10.590000,10.570,10.500,10.5600,10.570,10.500,...,10.630,10.69,10.720,10.690,10.690,10.690,10.690,10.690,10.690,NaN
1,AADI,1,5.760,5.3800,5.360000,5.330,5.250,5.2600,5.660,5.730,...,4.200,4.23,4.220,4.180,3.990,3.960,4.090,3.940,4.290,4.2200
2,AAL,2,16.760,16.5700,16.790001,16.750,16.230,15.9500,15.980,15.840,...,12.060,11.72,11.940,11.940,11.360,11.450,11.080,11.290,11.210,11.0400
3,AAME,3,1.940,1.8700,1.830000,1.830,1.770,1.7600,1.810,1.920,...,1.860,1.85,1.820,1.730,1.800,1.930,1.810,1.940,1.940,1.8000
4,AAOI,4,7.950,6.1900,6.710000,6.750,7.030,7.3900,6.590,11.010,...,7.950,7.55,7.650,7.590,7.550,7.450,6.910,7.170,7.440,7.1200
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2947,ZVRA,2947,4.710,4.6400,4.830000,4.880,4.900,4.8100,4.840,4.820,...,4.560,4.47,4.460,4.530,4.370,4.370,4.210,4.290,4.370,4.3900
2948,ZVSA,2948,0.166,0.1620,0.169000,0.173,0.170,0.1700,0.160,0.170,...,0.131,0.13,0.126,0.122,0.128,0.128,0.125,0.118,0.116,0.1145
2949,ZYME,2949,7.510,7.2800,7.270000,7.460,7.450,7.3200,7.320,7.390,...,6.420,6.50,7.050,7.250,7.160,7.210,7.190,7.000,7.010,6.9700
2950,ZYNE,2950,0.364,0.3551,0.359800,0.365,0.355,0.3688,0.352,0.353,...,1.300,1.30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [106]:
x = df.iloc[:, 2:]
i_split = 0
# 日付の特定
for i in range(len(x.columns)):
    date1 = '2023-09-20'

    # 日付をdatetimeオブジェクトに変換する
    date1_obj = datetime.strptime(x.columns[i], '%Y-%m-%d')
    date2_obj = datetime.strptime(date1, '%Y-%m-%d')

    if date1_obj > date2_obj:
        print(x.columns[i])
        print(i)
        i_split = i
        break

2023-09-21
40


In [107]:
# x = df.iloc[:, i_split-60:]
label = df['encode_name']

In [108]:
label = df['encode_name']

In [109]:
x

,2023-07-26,2023-07-27,2023-07-28,2023-07-31,2023-08-01,2023-08-02,2023-08-03,2023-08-04,2023-08-07,2023-08-08,...,2023-10-12,2023-10-13,2023-10-16,2023-10-17,2023-10-18,2023-10-19,2023-10-20,2023-10-23,2023-10-24,2023-10-25
0,10.570,10.5700,10.590000,10.570,10.500,10.5600,10.570,10.500,10.5000,10.5000,...,10.630,10.69,10.720,10.690,10.690,10.690,10.690,10.690,10.690,NaN
1,5.760,5.3800,5.360000,5.330,5.250,5.2600,5.660,5.730,5.9900,5.8400,...,4.200,4.23,4.220,4.180,3.990,3.960,4.090,3.940,4.290,4.2200
2,16.760,16.5700,16.790001,16.750,16.230,15.9500,15.980,15.840,15.9900,16.1800,...,12.060,11.72,11.940,11.940,11.360,11.450,11.080,11.290,11.210,11.0400
3,1.940,1.8700,1.830000,1.830,1.770,1.7600,1.810,1.920,1.8600,1.8200,...,1.860,1.85,1.820,1.730,1.800,1.930,1.810,1.940,1.940,1.8000
4,7.950,6.1900,6.710000,6.750,7.030,7.3900,6.590,11.010,12.3800,12.1900,...,7.950,7.55,7.650,7.590,7.550,7.450,6.910,7.170,7.440,7.1200
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2947,4.710,4.6400,4.830000,4.880,4.900,4.8100,4.840,4.820,4.8600,5.2000,...,4.560,4.47,4.460,4.530,4.370,4.370,4.210,4.290,4.370,4.3900
2948,0.166,0.1620,0.169000,0.173,0.170,0.1700,0.160,0.170,0.1640,0.1610,...,0.131,0.13,0.126,0.122,0.128,0.128,0.125,0.118,0.116,0.1145
2949,7.510,7.2800,7.270000,7.460,7.450,7.3200,7.320,7.390,7.2600,7.2900,...,6.420,6.50,7.050,7.250,7.160,7.210,7.190,7.000,7.010,6.9700
2950,0.364,0.3551,0.359800,0.365,0.355,0.3688,0.352,0.353,0.3421,0.3463,...,1.300,1.30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [110]:
num_cols = len(x.columns)
grouped_cols = [x.iloc[:, i:i+59] for i in range(0, num_cols) if len(x.iloc[:, i:i+59].columns) == 59]
# 各グループを表示
grouped_cols2 = []
for group in grouped_cols:
    df_combined = pd.concat([label, group], axis=1, ignore_index=False)
    df_combined = df_combined.dropna() 
    grouped_cols2.append(df_combined)
    print(df_combined.shape)

(2898, 60)
(2898, 60)
(2896, 60)
(2895, 60)
(2893, 60)
(2893, 60)
(2782, 60)


In [111]:
grouped_cols = grouped_cols2

In [112]:
grouped_cols2[-1]

,encode_name,2023-08-03,2023-08-04,2023-08-07,2023-08-08,2023-08-09,2023-08-10,2023-08-11,2023-08-14,2023-08-15,...,2023-10-12,2023-10-13,2023-10-16,2023-10-17,2023-10-18,2023-10-19,2023-10-20,2023-10-23,2023-10-24,2023-10-25
1,1,5.660000,5.730000,5.990000,5.840000,5.440000,5.780000,5.910000,5.940000,5.95,...,4.200000,4.23,4.220000,4.180,3.990000,3.960000,4.090000,3.940000,4.290000,4.2200
2,2,15.980000,15.840000,15.990000,16.180000,15.760000,15.760000,15.650000,15.880000,15.55,...,12.060000,11.72,11.940000,11.940,11.360000,11.450000,11.080000,11.290000,11.210000,11.0400
3,3,1.810000,1.920000,1.860000,1.820000,1.840000,1.770000,1.810000,1.970000,1.85,...,1.860000,1.85,1.820000,1.730,1.800000,1.930000,1.810000,1.940000,1.940000,1.8000
4,4,6.590000,11.010000,12.380000,12.190000,12.160000,14.440000,12.390000,14.840000,14.79,...,7.950000,7.55,7.650000,7.590,7.550000,7.450000,6.910000,7.170000,7.440000,7.1200
5,5,69.419998,65.333336,62.966667,64.260002,63.220001,62.900002,63.080002,64.160004,63.00,...,57.419998,56.16,57.630001,57.320,53.720001,52.849998,53.169998,53.540001,53.209999,53.3400
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2946,2946,6.370000,6.220000,6.320000,6.560000,6.700000,6.730000,6.840000,6.860000,6.80,...,6.330000,5.91,6.010000,5.900,5.610000,5.160000,4.620000,4.930000,5.120000,5.2600
2947,2947,4.840000,4.820000,4.860000,5.200000,4.950000,4.990000,5.200000,5.400000,5.23,...,4.560000,4.47,4.460000,4.530,4.370000,4.370000,4.210000,4.290000,4.370000,4.3900
2948,2948,0.160000,0.170000,0.164000,0.161000,0.159000,0.159000,0.165000,0.160000,0.16,...,0.131000,0.13,0.126000,0.122,0.128000,0.128000,0.125000,0.118000,0.116000,0.1145
2949,2949,7.320000,7.390000,7.260000,7.290000,7.700000,7.760000,7.170000,7.490000,7.42,...,6.420000,6.50,7.050000,7.250,7.160000,7.210000,7.190000,7.000000,7.010000,6.9700


In [113]:
list(grouped_cols[-1].loc[2950])

KeyError: 2950

In [114]:
grouped_cols[1].columns[-1]

'2023-10-18'

In [115]:
class LSTMClassifier(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(LSTMClassifier, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers

        # LSTM layer
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)

        # Fully connected layer
        self.fc = nn.Linear(hidden_size, num_classes)

        # 損失関数と最適化関数
        self.criterion = nn.CrossEntropyLoss()
        self.criterion_2 = nn.HingeEmbeddingLoss()
        self.optimizer = optim.Adam(self.parameters())

    def forward(self, x, hidden0=None):        
        # Forward pass
        out, (hidden, cell) = self.lstm(x, hidden0)
        # Index hidden state of last time step
        out = self.fc(out)
        return out

In [116]:
input_size = 60
hidden_size = 4
num_layers = 4
num_classes = 2

# Create the model
model = LSTMClassifier(input_size, hidden_size, num_layers, num_classes).to(device)
# 現在の結論として80epchの中で一番tain lossが小さいものを選択するのが良さそう
# model_weight_train_loss_0.35017_test_loss_1.18746_epoch_57.pth
# model_weight_train_loss_0.31690_test_loss_1.14792_epoch_71.pth 良い
# model_weight_train_loss_0.29460_test_loss_1.44028_epoch_79.pth 良い
# model_weight_train_loss_0.26548_test_loss_1.60183_epoch_99.pth 微妙
# model_weight_train_loss_0.28409_test_loss_1.72535_epoch_183.pth　微妙
model.load_state_dict(torch.load('../predict_model/weights/model_weight_train_loss_0.39769_test_loss_1.14234_epoch_36.pth', map_location="cpu"))

<All keys matched successfully>

In [117]:
grouped_cols[0].columns[-1]

'2023-10-17'

In [118]:
for number, group in enumerate(grouped_cols):
    group.to_csv(f'temp_{number}')

In [119]:
# 各グループを表示
for number, group in enumerate(grouped_cols):
    x = group
    # 2.DataFrameをnarrayに変換
    x = x.to_numpy().astype("float32")

    # 3.ndarrayをTensorに変換
    x = torch.from_numpy(x).to(device)

    # 4.TensorからDatasetを作成
    dataset = torch.utils.data.TensorDataset(x)

    # 6.DataLoaderに変換
    batch_size = 20
    test_loader = DataLoader(dataset, batch_size=len(dataset), shuffle=False)

    # モデルを評価モードにする
    model.eval()

    # 正しい予測数、全体のデータ数を数えるカウンターの0初期化
    total_correct = 0

    name_list = []
    index_y_list = []
    ans_list = []
    val_list = []

    loss_mean = 0
    m = nn.Softmax(dim=2)

    for j, x in enumerate(test_loader):
        x = torch.stack(x)  # リストをテンソルに変換
        y = model(x)  # 順伝播（=予測）
        
        # ミニバッチごとの正答率と損失を求める
        _, index_y = torch.max(y, axis=1)  # 最も確率が高いと予測したindex
        # print(index_y, index_t)
        res = m(y)
        val, index_y = torch.max(m(y)[0], axis=1)
        print(val.tolist(), index_y.tolist())
        for j, (i, ii) in enumerate(zip(val.tolist(), index_y.tolist())):
            if ii == 1:
                name_list.append(df['name'][j])
                index_y_list.append(ii)
                val_list.append(i)

    # リストをDataFrameに変換
    result_df = pd.DataFrame({
        'name': name_list,
        'ai_predict': index_y_list,
        'ai_confidence': val_list
    })

    result_df = result_df.sort_values(by='ai_confidence', ascending=False)

    """
    予測するのはcsvの最後の日付の次の日、ただ、その次の日は休日の可能性があるので、
    次のCSVの日付を予測日として保存する
    最後のcsvの場合は、最後の日付の次の日を予測日として保存する
    (このケースはエラーになる可能性があるので注意)
    """
    try:
        date_str = grouped_cols[number+1].columns[-1]
        date_obj = datetime.strptime(date_str, '%Y-%m-%d')
        new_date_str = date_obj.strftime('%Y-%m-%d')

    except Exception as e:
        date_str = group.columns[-1]
        date_obj = datetime.strptime(date_str, '%Y-%m-%d')
        new_date_obj = date_obj + timedelta(days=1)
        new_date_str = new_date_obj.strftime('%Y-%m-%d')

    print(new_date_str)
    print(val.tolist())
    # if new_date_str == '2023-10-25':
    #     print(list(y))

    result_df.to_csv(f'./ai_temp/{new_date_str}.csv')

[0.983115017414093, 0.9898061156272888, 0.9844847917556763, 0.9501435160636902, 0.8967752456665039, 0.9716904759407043, 0.9644771218299866, 0.9569385051727295, 0.9546009302139282, 0.9209256768226624, 0.9296225309371948, 0.9512959718704224, 0.9511633515357971, 0.9311947226524353, 0.6526269316673279, 0.9319249987602234, 0.815546452999115, 0.9574335813522339, 0.9525203704833984, 0.9477704167366028, 0.9475423693656921, 0.6521903872489929, 0.9543174505233765, 0.9444504380226135, 0.9454061985015869, 0.526893138885498, 0.7224134206771851, 0.9702658653259277, 0.9564499258995056, 0.949587881565094, 0.9486851692199707, 0.9434303641319275, 0.9464707374572754, 0.9476187229156494, 0.9474889636039734, 0.9475162625312805, 0.9475744962692261, 0.9475287199020386, 0.9470393061637878, 0.9468997716903687, 0.5572959184646606, 0.9515079259872437, 0.9513203501701355, 0.9478404521942139, 0.5758253931999207, 0.9561444520950317, 0.6656628847122192, 0.9304001927375793, 0.9577000737190247, 0.8819829821586609, 0.9

In [36]:
torch.max(m(y)[0], axis=1)

torch.return_types.max(
values=tensor([0.8844, 0.7197, 0.5934,  ..., 0.8206, 0.7313, 0.5458], device='cuda:0',
       grad_fn=<MaxBackward0>),
indices=tensor([1, 1, 1,  ..., 1, 1, 0], device='cuda:0'))

In [198]:
result_df

,name,ai_predict,ai_confidence
29,AEI,1,0.988075
28,ADTX,1,0.987467
19,ACRX,1,0.984830
37,AGFY,1,0.982886
22,ADD,1,0.982705
...,...,...,...
1171,SGH,1,0.501737
1393,VC,1,0.501423
1330,TOMZ,1,0.501298
570,HLIT,1,0.500268


In [26]:
df['0'][0]

'AADI'

In [5]:
df.iloc[:, 2:]

,2,3,4,5,6,7,8,9,10,11,...,51,52,53,54,55,56,57,58,59,60
0,0.944132,0.912860,0.946943,0.958538,0.950808,0.940970,0.956079,0.921644,0.936753,0.975755,...,0.892481,0.904076,0.907238,0.880534,0.873507,0.908292,0.864020,0.846451,0.837316,0.873858
1,0.906110,0.901617,0.904313,0.911500,0.880952,0.882300,0.897574,0.891285,0.877359,0.876909,...,0.742138,0.761456,0.777179,0.814016,0.820305,0.820305,0.816262,0.832884,0.810871,0.811770
2,0.960089,0.995565,0.997783,0.971175,0.955654,0.975610,0.960089,0.942350,0.920177,0.942350,...,0.758315,0.789357,0.691796,0.691796,0.687361,0.711752,0.678492,0.694013,0.638581,0.616408
3,0.445736,0.438631,0.439922,0.436047,0.465116,0.474160,0.466408,0.462532,0.479328,0.478682,...,0.351421,0.366279,0.347545,0.353359,0.352713,0.364341,0.361111,0.354005,0.342377,0.345607
4,0.644453,0.636428,0.626557,0.619085,0.621760,0.639195,0.645745,0.653586,0.659213,0.650726,...,0.716707,0.721151,0.713096,0.721336,0.727724,0.727447,0.738927,0.734668,0.738001,0.733002
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3531,0.616279,0.688631,0.590439,0.574289,0.495478,0.560078,0.524548,0.510982,0.550388,0.544574,...,0.635659,0.601421,0.618863,0.606589,0.629845,0.664083,0.711886,0.708656,0.728036,0.666667
3532,0.910362,0.914211,0.933452,0.974470,0.984607,0.971654,0.960860,0.948376,0.956355,0.949597,...,0.862988,0.798990,0.802796,0.804769,0.782779,0.801668,0.810549,0.801668,0.803923,0.781651
3533,0.965996,0.969916,0.970680,0.987478,0.986154,0.993128,0.983100,0.977043,0.981166,0.985594,...,0.894513,0.886561,0.890944,0.897520,0.876519,0.868720,0.870046,0.872696,0.885643,0.878762
3534,0.669834,0.687958,0.675013,0.692581,0.726424,0.747137,0.735301,0.735301,0.755459,0.751576,...,0.705712,0.691842,0.687588,0.699794,0.688698,0.688513,0.698725,0.712837,0.701325,0.686656


In [6]:
# 1.DataFrameを説明変数と目的変数に分ける
x = df.iloc[:, 2:]
t = pd.get_dummies(df['1'])


In [7]:
x


,2,3,4,5,6,7,8,9,10,11,...,51,52,53,54,55,56,57,58,59,60
0,0.944132,0.912860,0.946943,0.958538,0.950808,0.940970,0.956079,0.921644,0.936753,0.975755,...,0.892481,0.904076,0.907238,0.880534,0.873507,0.908292,0.864020,0.846451,0.837316,0.873858
1,0.906110,0.901617,0.904313,0.911500,0.880952,0.882300,0.897574,0.891285,0.877359,0.876909,...,0.742138,0.761456,0.777179,0.814016,0.820305,0.820305,0.816262,0.832884,0.810871,0.811770
2,0.960089,0.995565,0.997783,0.971175,0.955654,0.975610,0.960089,0.942350,0.920177,0.942350,...,0.758315,0.789357,0.691796,0.691796,0.687361,0.711752,0.678492,0.694013,0.638581,0.616408
3,0.445736,0.438631,0.439922,0.436047,0.465116,0.474160,0.466408,0.462532,0.479328,0.478682,...,0.351421,0.366279,0.347545,0.353359,0.352713,0.364341,0.361111,0.354005,0.342377,0.345607
4,0.644453,0.636428,0.626557,0.619085,0.621760,0.639195,0.645745,0.653586,0.659213,0.650726,...,0.716707,0.721151,0.713096,0.721336,0.727724,0.727447,0.738927,0.734668,0.738001,0.733002
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3531,0.616279,0.688631,0.590439,0.574289,0.495478,0.560078,0.524548,0.510982,0.550388,0.544574,...,0.635659,0.601421,0.618863,0.606589,0.629845,0.664083,0.711886,0.708656,0.728036,0.666667
3532,0.910362,0.914211,0.933452,0.974470,0.984607,0.971654,0.960860,0.948376,0.956355,0.949597,...,0.862988,0.798990,0.802796,0.804769,0.782779,0.801668,0.810549,0.801668,0.803923,0.781651
3533,0.965996,0.969916,0.970680,0.987478,0.986154,0.993128,0.983100,0.977043,0.981166,0.985594,...,0.894513,0.886561,0.890944,0.897520,0.876519,0.868720,0.870046,0.872696,0.885643,0.878762
3534,0.669834,0.687958,0.675013,0.692581,0.726424,0.747137,0.735301,0.735301,0.755459,0.751576,...,0.705712,0.691842,0.687588,0.699794,0.688698,0.688513,0.698725,0.712837,0.701325,0.686656


In [8]:
t

,0,1
0,1,0
1,1,0
2,1,0
3,1,0
4,1,0
...,...,...
3531,1,0
3532,0,1
3533,1,0
3534,1,0


In [9]:
# 2.DataFrameをnarrayに変換
x = x.to_numpy().astype("float32")

# 3.ndarrayをTensorに変換
x = torch.from_numpy(x).to(device)

# 4.TensorからDatasetを作成
dataset = torch.utils.data.TensorDataset(x)

# 6.DataLoaderに変換
batch_size = 20
test_loader = DataLoader(dataset, batch_size=len(dataset), shuffle=False)

In [94]:
class Net2(nn.Module):
    def __init__(self):
        super().__init__()

        self.rnn = nn.LSTM(input_size = 64,
                            hidden_size = 100,
                            batch_first = True)
        self.output_layer = nn.Linear(20, 2)

        # 損失関数と最適化関数
        self.criterion = nn.CrossEntropyLoss()
        self.criterion_2 = nn.HingeEmbeddingLoss()
        self.optimizer = optim.Adam(self.parameters())

    def forward(self, inputs, hidden0=None):
        output, (hidden, cell) = self.rnn(inputs, hidden0) #LSTM層
        output = self.output_layer(output[:, -1]) 
        return output

In [95]:
class LSTMClassifier(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(LSTMClassifier, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers

        # LSTM layer
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)

        # Fully connected layer
        self.fc = nn.Linear(hidden_size, num_classes)

        # 損失関数と最適化関数
        self.criterion = nn.CrossEntropyLoss()
        self.criterion_2 = nn.HingeEmbeddingLoss()
        self.optimizer = optim.Adam(self.parameters())

    def forward(self, x, hidden0=None):        
        # Forward pass
        out, (hidden, cell) = self.lstm(x, hidden0)
        # Index hidden state of last time step
        out = self.fc(out)
        return out

In [99]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()

        # 全結合層を6つに増やす
        self.fc1 = nn.Linear(64, 32)
        self.dropout1 = nn.Dropout(0.3)  # ドロップアウトを追加
        self.fc2 = nn.Linear(32, 64)
        self.dropout2 = nn.Dropout(0.3)  # ドロップアウトを追加
        self.fc3 = nn.Linear(64, 64)
        self.dropout3 = nn.Dropout(0.3)  # ドロップアウトを追加
        self.fc4 = nn.Linear(64, 64)
        self.dropout4 = nn.Dropout(0.3)  # ドロップアウトを追加
        self.fc5 = nn.Linear(64, 32)
        self.dropout5 = nn.Dropout(0.3)  # ドロップアウトを追加
        self.fc6 = nn.Linear(32, 2)

        # 損失関数と最適化関数
        self.criterion = nn.CrossEntropyLoss()
        self.criterion_2 = nn.HingeEmbeddingLoss()
        self.optimizer = optim.Adam(self.parameters())

    def forward(self, x):
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout1(x)  # ドロップアウトを適用
        x = self.fc2(x)
        x = F.relu(x)
        x = self.dropout2(x)  # ドロップアウトを適用
        x = self.fc3(x)
        x = F.relu(x)
        x = self.dropout3(x)  # ドロップアウトを適用
        x = self.fc4(x)
        x = F.relu(x)
        x = self.dropout4(x)  # ドロップアウトを適用
        x = self.fc5(x)
        x = F.relu(x)
        x = self.dropout5(x)  # ドロップアウトを適用
        x = self.fc6(x)
        return x

In [100]:
def test(model, data_loader):
    # モデルを評価モードにする
    model.eval()

    # 正しい予測数、全体のデータ数を数えるカウンターの0初期化
    total_data_len = 0
    total_correct = 0

    loss_mean = 0

    for j, (x, t) in enumerate(data_loader):
        y = model(x)  # 順伝播（=予測）
        loss = model.criterion(y, t)  # 損失を計算
        loss_mean += loss.item()

        # ミニバッチごとの正答率と損失を求める
        _, index_y = torch.max(y, axis=1)  # 最も確率が高いと予測したindex
        _, index_t = torch.max(t, axis=1)  # 正解のindex
        for i in range(len(t)):  # データ一つずつループ,ミニバッチの中身出しきるまで
            total_data_len += 1  # 全データ数を集計
            if index_y[i] == index_t[i]:
                total_correct += 1 # 正解のデータ数を集計

    loss_mean = loss_mean / (j+1)

    return total_correct, total_data_len, loss_mean

In [101]:
input_size = 59
hidden_size = 2
num_layers = 3
num_classes = 2

# Create the model
model = LSTMClassifier(input_size, hidden_size, num_layers, num_classes).to(device)
model.load_state_dict(torch.load('../predict_model/weights/model_weight3.pth', map_location="cpu"))


<All keys matched successfully>

In [17]:
# モデルを評価モードにする
model.eval()

# 正しい予測数、全体のデータ数を数えるカウンターの0初期化
total_data_len = 0
total_correct = 0

name_list = []
index_y_list = []
ans_list = []
val_list = []

loss_mean = 0
m = nn.Softmax(dim=2)

for j, x in enumerate(test_loader):
    x = torch.stack(x)  # リストをテンソルに変換
    y = model(x)  # 順伝播（=予測）
    # print(m(y))
    
    
    # ミニバッチごとの正答率と損失を求める
    _, index_y = torch.max(y, axis=1)  # 最も確率が高いと予測したindex
    # print(index_y, index_t)
    for i in range(len(t)):  # データ一つずつループ,ミニバッチの中身出しきるまで
        total_data_len += 1  # 全データ数を集計
    res = m(y)
    val, index_y = torch.max(m(y)[0], axis=1)
    print(val.tolist(), index_y.tolist())
    for j, (i, ii) in enumerate(zip(val.tolist(), index_y.tolist())):
        name_list.append(df['0'][j])
        index_y_list.append(ii)
        val_list.append(i)

[0.6188809275627136, 0.5900458097457886, 0.7202277779579163, 0.6217108964920044, 0.6510367393493652, 0.5682734847068787, 0.6661288738250732, 0.6205351948738098, 0.6994403004646301, 0.6248250603675842, 0.741827130317688, 0.6049798130989075, 0.6640167832374573, 0.5857458710670471, 0.6653698086738586, 0.5793137550354004, 0.7054375410079956, 0.5996900200843811, 0.741631805896759, 0.6484270095825195, 0.6529103517532349, 0.6232622861862183, 0.661768913269043, 0.638318657875061, 0.6749601364135742, 0.6101622581481934, 0.7435817718505859, 0.5827338695526123, 0.6395730376243591, 0.6243159174919128, 0.6500524282455444, 0.630545973777771, 0.6717182397842407, 0.6300795674324036, 0.744522750377655, 0.6389649510383606, 0.6072825789451599, 0.6068164110183716, 0.639148473739624, 0.6837903261184692, 0.6729153990745544, 0.6194990277290344, 0.7362712621688843, 0.6448964476585388, 0.6151716113090515, 0.615625262260437, 0.6318235993385315, 0.7192761301994324, 0.6588127017021179, 0.6537846922874451, 0.74664

In [43]:
# リストをDataFrameに変換
result_df = pd.DataFrame({
    'name': name_list,
    'index_y': index_y_list,
    'ai_confidence': val_list
})

In [46]:
# 7つずつに分割する
result_df_split = np.array_split(result_df, len(result_df) // interval + 1)

# 分割したデータフレームを表示する
for df in result_df_split:
    # val列で降順にソート
    sorted_df = df.sort_values(by='ai_confidence', ascending=False)
    print(sorted_df.head(1))

   name  index_y  ai_confidence
2  AAME        0       0.720228
    name  index_y  ai_confidence
10  AAME        0       0.741827
    name  index_y  ai_confidence
18  AAME        0       0.741632
    name  index_y  ai_confidence
26  AAME        0       0.743582
    name  index_y  ai_confidence
34  AAME        0       0.744523
    name  index_y  ai_confidence
42  AAME        0       0.736271
    name  index_y  ai_confidence
50  AAME        0       0.746645
    name  index_y  ai_confidence
63  ABCL        0       0.717605
    name  index_y  ai_confidence
66  AAME        0        0.73218
    name  index_y  ai_confidence
74  AAME        0       0.718493
    name  index_y  ai_confidence
82  AAME        0       0.722851
    name  index_y  ai_confidence
90  AAME        0       0.719773
    name  index_y  ai_confidence
99  AAOI        0       0.768365
     name  index_y  ai_confidence
107  AAOI        0       0.754259
     name  index_y  ai_confidence
114  AAME        0       0.732721
     nam

In [19]:
result_df

,name,index_y,val
0,AADI,0,0.618881
1,AAL,0,0.590046
2,AAME,0,0.720228
3,AAOI,0,0.621711
4,AAON,1,0.651037
...,...,...,...
3531,AAOI,1,0.828456
3532,AAON,0,0.710090
3533,AAPL,0,0.703670
3534,ABCB,0,0.554590


In [130]:
# val列で降順にソート
result_df = result_df.sort_values(by='val', ascending=False)

# DataFrameの内容を表示
print(result_df)

In [134]:
# csv_path = "pread_AI.csv"
# headers = ','.join(t.columns.to_list())
# with open(csv_path, "w", encoding="utf_8") as csv_file:
#     csv_file.write(headers + '\n')
pred_list = []
for data_raw in dataset:
    data = torch.Tensor(data_raw[0]).to(device)
    pred = model(data)
    pred = F.softmax(pred, dim=0)
    pred_list.append(pred.tolist())

    # with open(csv_path, 'a') as file:
    #     writer = csv.writer(file)
    #     writer.writerow(pred_list)